# Lista 07 - Aprendizado de Máquina

## Aluno: Ruan Felipe de Almeida Silva - 397048

In [159]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import datasets

#### Avalie o desempenho do algoritmo kNN para o dataset "Pima Indians Diabetes Database". Separe os dados em treino (80%) e teste (20%).


In [120]:
# dataframe = pd.read_csv('/content/drive/MyDrive/ufc - ec/Aprend_Maquina/datasets/diabetes.csv', delimiter=',')
dataframe = pd.read_csv('diabetes.csv', delimiter=',')
dataframe.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [121]:
dataframe.shape

(768, 9)

In [122]:
dataframe.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [123]:
dataframe.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [124]:
dataframe.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [125]:
X = dataframe.iloc[:,:-1]
y = dataframe.iloc[:,-1]
np.unique(y)

array([0, 1])

In [126]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [127]:
X_values = X.values
y_values = y.values
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_values, y_values, test_size=0.2, random_state=142, stratify=y_values)
print("Conjunto x de treino:", len(X_train))
print("Conjunto x de teste:", len(X_test))
print("Conjunto y de treino:", len(y_train))
print("Conjunto y de teste:", len(y_test))

Conjunto x de treino: 614
Conjunto x de teste: 154
Conjunto y de treino: 614
Conjunto y de teste: 154


#### 1. Use validação holdout para obter o melhor valor de k para o kNN com e sem escalonamento de features. Para isso, divida o conjunto de treino (80%) entre treino_holdout (70% do conjunto de treino) e validacao_holdout (30% do conjunto de treino). Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação (validacao_holdout). Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

* Dicas: (1) use o Pipeline do scikit learn para realizar o escalonamento de features. (2) Não use nessa questão nem o cross_val_score, nem o GridSearchCV do scikit learn.

In [128]:
X_train_holdout, X_validacao_holdout, y_train_holdout, y_validacao_holdout = model_selection.train_test_split(X_train, y_train, test_size=0.3, random_state=142, stratify=y_train)
print("Conjunto x de treino holdout:", len(X_train_holdout))
print("Conjunto x de validação holdout:", len(X_validacao_holdout))
print("Conjunto y de treino holdout:", len(y_train_holdout))
print("Conjunto y de validação holdout", len(y_validacao_holdout))

Conjunto x de treino holdout: 429
Conjunto x de validação holdout: 185
Conjunto y de treino holdout: 429
Conjunto y de validação holdout 185


In [129]:
array_k = np.arange(1, 31)

std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()

scalers = ['no scaler', std, norm]
results = []

for k in array_k:
    for scaler in scalers:
        knn = KNeighborsClassifier(n_neighbors=k)
        if scaler == 'no scaler':
            pipeline = Pipeline([('estimator', knn)])
            scl = 'No Scaler'
        else:
            pipeline = Pipeline([('transformer', scaler), ('estimator', knn)])
            s = str(scaler)
            ind_parenteses = s.find('(')
            scl = s[:ind_parenteses]
            

        pipeline.fit(X_train_holdout, y_train_holdout)
        y_pred = pipeline.predict(X_validacao_holdout)
        accuracy = metrics.accuracy_score(y_validacao_holdout, y_pred)
        f1score = metrics.f1_score(y_validacao_holdout, y_pred, average='weighted', zero_division=0)
        
        print(f'Escalonamento: {scl}')
        print(f'kNN: k = {k}')
        print('Acurácia: {:.5f}'.format(accuracy))
        print('F1-Score: {:.5f}'.format(f1score))        
        print("\n---------------------------------------------------------------------------------\n")

        
        resultado = {'Value of k':k, 'Scaler':scl, 'Accuracy':accuracy, 'F1-Score':f1score}
        results.append(resultado)

Escalonamento: No Scaler
kNN: k = 1
Acurácia: 0.69189
F1-Score: 0.69349

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 1
Acurácia: 0.74054
F1-Score: 0.73325

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 1
Acurácia: 0.72432
F1-Score: 0.71884

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 2
Acurácia: 0.72432
F1-Score: 0.68987

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 2
Acurácia: 0.71892
F1-Score: 0.67497

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 2
Acurácia: 0.70270
F1-Score: 0.66554

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 3
Acurácia: 0.

In [130]:
df_results = pd.DataFrame(results)
df_results.sort_values(by='F1-Score', ascending=False)

,Value of k,Scaler,Accuracy,F1-Score
14,5,MinMaxScaler,0.800000,0.793794
38,13,MinMaxScaler,0.794595,0.787599
68,23,MinMaxScaler,0.789189,0.778489
56,19,MinMaxScaler,0.783784,0.772016
32,11,MinMaxScaler,0.778378,0.771501
...,...,...,...,...
10,4,StandardScaler,0.724324,0.689866
3,2,No Scaler,0.724324,0.689866
9,4,No Scaler,0.702703,0.685398
4,2,StandardScaler,0.718919,0.674972


In [131]:
best_params = df_results.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k               5
Scaler        MinMaxScaler
Accuracy               0.8
F1-Score          0.793794
Name: 14, dtype: object

In [132]:
norm = preprocessing.MinMaxScaler()

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
pipeline = Pipeline([('transformer', norm), ('estimator', knn_melhor)])
pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score))        


Escalonamento: MinMaxScaler
kNN: k = 5
Acurácia: 0.69481
F1-Score: 0.69661


In [133]:
df_noScaler = df_results[df_results['Scaler'] == 'No Scaler']
df_noScaler.sort_values(by='F1-Score', ascending=False).head(10)

,Value of k,Scaler,Accuracy,F1-Score
30,11,No Scaler,0.772973,0.765240
21,8,No Scaler,0.767568,0.757397
24,9,No Scaler,0.762162,0.756803
18,7,No Scaler,0.756757,0.754275
42,15,No Scaler,0.756757,0.747711
36,13,No Scaler,0.756757,0.747711
27,10,No Scaler,0.756757,0.744410
78,27,No Scaler,0.751351,0.739614
48,17,No Scaler,0.745946,0.734829
72,25,No Scaler,0.745946,0.733051


In [134]:
best_params = df_noScaler.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k           11
Scaler        No Scaler
Accuracy       0.772973
F1-Score        0.76524
Name: 30, dtype: object

In [135]:

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
knn_melhor.fit(X_train, y_train)


y_pred = knn_melhor.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score))    

Escalonamento: No Scaler
kNN: k = 11
Acurácia: 0.71429
F1-Score: 0.70451


#### 2. Use validação cruzada com k-Fold=5 para obter o melhor valor de k para o kNN com e sem escalonamento de features. Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação. Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

* Dicas: (1) use a função cross_val_score do scikit learn com cv=5. (2) use o Pipeline do scikit learn para realizar o escalonamento de features. (3) Não use nessa questão o GridSearchCV do scikit learn.

In [136]:
array_k = np.arange(1, 31)

std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()

scalers = ['no scaler', std, norm]
results = []

for k in array_k:
    for scaler in scalers:
        knn = KNeighborsClassifier(n_neighbors=k)
        if scaler == 'no scaler':
            pipeline = Pipeline([('estimator', knn)])
            scl = 'No Scaler'
        else:
            pipeline = Pipeline([('transformer', scaler), ('estimator', knn)])
            s = str(scaler)
            ind_parenteses = s.find('(')
            scl = s[:ind_parenteses]
            

        f1score = model_selection.cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_weighted')
        
        print(f'Escalonamento: {scl}')
        print(f'kNN: k = {k}')
        print(f'F1-Score: {f1score}')        
        print("\n---------------------------------------------------------------------------------\n")

        
        resultado = {'Value of k':k, 'Scaler':scl, 'F1-Score':f1score.mean()}
        results.append(resultado)

Escalonamento: No Scaler
kNN: k = 1
F1-Score: [0.66493902 0.6939615  0.72357724 0.70163091 0.60856421]

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 1
F1-Score: [0.69684959 0.73666468 0.71292322 0.7375914  0.73742167]

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 1
F1-Score: [0.70513956 0.72749172 0.75469953 0.74722495 0.69049584]

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 2
F1-Score: [0.70790481 0.68598813 0.69405851 0.69705182 0.64048092]

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 2
F1-Score: [0.66660744 0.67636369 0.7148648  0.69069014 0.70011774]

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 2
F1-Score: [0.

In [137]:
df_results = pd.DataFrame(results)
df_results.sort_values(by='F1-Score', ascending=False)

,Value of k,Scaler,F1-Score
19,7,StandardScaler,0.782250
38,13,MinMaxScaler,0.765790
25,9,StandardScaler,0.760645
20,7,MinMaxScaler,0.755049
32,11,MinMaxScaler,0.753592
...,...,...,...
9,4,No Scaler,0.703926
5,2,MinMaxScaler,0.693928
4,2,StandardScaler,0.689729
3,2,No Scaler,0.685097


In [138]:
best_params = df_results.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k                 7
Scaler        StandardScaler
F1-Score             0.78225
Name: 19, dtype: object

In [139]:
std = preprocessing.StandardScaler()

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
pipeline = Pipeline([('transformer', std), ('estimator', knn_melhor)])
pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score)) 

Escalonamento: StandardScaler
kNN: k = 7
Acurácia: 0.71429
F1-Score: 0.71159


In [140]:
df_noScaler = df_results[df_results['Scaler'] == 'No Scaler']
df_noScaler.sort_values(by='F1-Score', ascending=False).head(10)

,Value of k,Scaler,F1-Score
36,13,No Scaler,0.748976
24,9,No Scaler,0.744879
18,7,No Scaler,0.743514
33,12,No Scaler,0.740375
27,10,No Scaler,0.737264
39,14,No Scaler,0.737213
21,8,No Scaler,0.734474
78,27,No Scaler,0.732170
72,25,No Scaler,0.730606
42,15,No Scaler,0.730599


In [141]:
best_params = df_noScaler.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k           13
Scaler        No Scaler
F1-Score       0.748976
Name: 36, dtype: object

In [142]:

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
knn_melhor.fit(X_train, y_train)


y_pred = knn_melhor.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score)) 

Escalonamento: No Scaler
kNN: k = 13
Acurácia: 0.70779
F1-Score: 0.70080


#### 3. Responda a questão 1 só que agora usando o GridSearchCV do scikit learn. Dica: Nesta questão você não deve usar o cross_val_score do scikit learn.

In [169]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_values, y_values, test_size=0.2, random_state=142, stratify=y_values)
print("Conjunto x de treino:", len(X_train))
print("Conjunto x de teste:", len(X_test))
print("Conjunto y de treino:", len(y_train))
print("Conjunto y de teste:", len(y_test))

Conjunto x de treino: 614
Conjunto x de teste: 154
Conjunto y de treino: 614
Conjunto y de teste: 154


In [173]:
std = preprocessing.StandardScaler()
model = KNeighborsClassifier()
pipe = pipeline.make_pipeline(std, model)
param_k = np.arange(1, 11)
param_grid = [{'n_neighbors':param_k,
               'weights':['uniform', 'distance']}]

gridSearch = model_selection.GridSearchCV(estimator=pipe, iid=False, param_grid=param_grid, return_train_score=False, scoring='f1_weighted', cv=5)
gridSearch.fit(X=X_train, y=y_train)
print(gridSearch.best_score)
print(gridSearch.best_params)


ValueError: Invalid parameter n_neighbors for estimator Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())]). Check the list of available parameters with `estimator.get_params().keys()`.